In [3]:
import cv2
import numpy as np
import random
import os
from PIL import Image
import time
import smtplib
import os


In [4]:

net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
distance_thres = 50

count = 0


cap = cv2.VideoCapture(0)
def dist(pt1,pt2):
    try:
        return ((pt1[0]-pt2[0])**2 + (pt1[1]-pt2[1])**2)**0.5
    except:
        return
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
print('Output layers',output_layers)
_,frame = cap.read()
fourcc = cv2.VideoWriter_fourcc(*"MJPG")
writer = cv2.VideoWriter('output.avi', fourcc, 30,(frame.shape[1], frame.shape[0]), True)
ret = True
while ret:
    ret, img = cap.read()
    if ret:
        height, width = img.shape[:2]
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)
        confidences = []
        boxes = []
        
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                if class_id!=0:
                    continue
                confidence = scores[class_id]
                if confidence > 0.3:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        persons = []
        person_centres = []
        violate = set()
        for i in range(len(boxes)):
            if i in indexes:
                x,y,w,h = boxes[i]
                persons.append(boxes[i])
                person_centres.append([x+w//2,y+h//2])
        
        v = 0
        for (x,y,w,h) in persons:
            color = (0,255,0)
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            cv2.circle(img,(x+w//2,y+h//2),2,(0,0,255),2)

            my_email = 'your email'
            my_pass = 'your pass'
        
    
            if count == 0:
                
                ### Sending email alert that someone has entered office.
                with smtplib.SMTP("smtp.gmail.com") as connection:
                    connection.starttls()
                    connection.login(user=my_email, password=my_pass)
                    connection.sendmail(from_addr=my_email,
                            to_addrs="abc@example.com",
                            msg="Subject: Person alert!!!!!!\n\nHuman detected and" 
                            " look at console for mail confirmation!!!!!")
                    print("Mail has been sent successfully!")  

            count = count+1 


        writer.write(img)
        cv2.imshow("Image", img)
    
    if cv2.waitKey(1) == 27:
        break


cap.release()
cv2.destroyAllWindows()

Output layers ['yolo_82', 'yolo_94', 'yolo_106']


UnicodeEncodeError: 'ascii' codec can't encode characters in position 38-41: ordinal not in range(128)